In [30]:
#using PyCall
using DataFrames
using CSV
using PyCall, JLD, PyCallJLD

In [5]:
## Example of python code used into julia
#ipfml_utils = pyimport("ipfml.utils")
#b = [1:10;]
#ipfml_utils.get_entropy(b)

0.83418762482611

## Example of Julia use for train

### Prepare dataframe

In [6]:
## Define file variable
data_filename = "../data/test_data"
output_filename = "test_model"

"test_model"

In [7]:
dataset_train = CSV.read("$data_filename.train", nastrings=["NA", "na", "n/a", "missing"], delim=';', header=0)
dataset_test = CSV.read("$data_filename.test", nastrings=["NA", "na", "n/a", "missing"], delim=';', header=0)
first(dataset_train, 3)

,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9,Column10,Column11,Column12,Column13,Column14,Column15,Column16,Column17,Column18,Column19,Column20,Column21,Column22,Column23,Column24,Column25,Column26,Column27,Column28,Column29,Column30,Column31,Column32,Column33,Column34,Column35,Column36,Column37,Column38,Column39,Column40,Column41,Column42,Column43,Column44,Column45,Column46,Column47,Column48,Column49,Column50,Column51,Column52,Column53,Column54,Column55,Column56,Column57,Column58,Column59,Column60,Column61,Column62,Column63,Column64,Column65,Column66,Column67,Column68,Column69,Column70,Column71,Column72,Column73,Column74,Column75,Column76,Column77,Column78,Column79,Column80,Column81,Column82,Column83,Column84,Column85,Column86,Column87,Column88,Column89,Column90,Column91,Column92,Column93,Column94,Column95,Column96,Column97,Column98,Column99,Column100,Column101,Column102,Column103,Column104,Column105,Column106,Column107,Column108,Column109,Column110,Column111,Column112,Column113,Column114,Column115,Column116,Column117,Column118,Column119,Column120,Column121,Column122,Column123,Column124,Column125,Column126,Column127,Column128,Column129,Column130,Column131,Column132,Column133,Column134,Column135,Column136,Column137,Column138,Column139,Column140,Column141,Column142,Column143,Column144,Column145,Column146,Column147,Column148,Column149,Column150,Column151,Column152,Column153,Column154,Column155,Column156,Column157,Column158,Column159,Column160,Column161,Column162,Column163,Column164,Column165,Column166,Column167,Column168,Column169,Column170,Column171,Column172,Column173,Column174,Column175,Column176,Column177,Column178,Column179,Column180,Column181,Column182,Column183,Column184,Column185,Column186,Column187,Column188,Column189,Column190,Column191,Column192,Column193,Column194,Column195,Column196,Column197,Column198,Column199,Column200,Column201
,Int64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰
1,1,0.808874,0.0262689,0.0222177,0.015871,0.0128175,0.0104919,0.00962279,0.00881732,0.00835705,0.0080237,0.00758001,0.00727214,0.00676085,0.00669725,0.00631513,0.0057416,0.00562775,0.00539497,0.00510166,0.00492374,0.00467361,0.00458811,0.0044562,0.00411901,0.00403369,0.00376357,0.00372016

In [8]:
nb_columns = size(dataset_train, 2)
train_labels = dataset_train[1]
train_data = dataset_train[2:nb_columns]
print("Split datasets done..")

Split datasets done..

In [9]:
noisy_df_train = filter(row -> row[1] == 1, dataset_train)
not_noisy_df_train = filter(row -> row[1] == 0, dataset_train)
nb_noisy_train = size(noisy_df_train, 1)

597

In [10]:
noisy_df_test = filter(row -> row[1] == 1, dataset_test)
not_noisy_df_test = filter(row -> row[1] == 0, dataset_test)
nb_noisy_test = size(noisy_df_test, 1)

357

In [11]:
#final_df_train = join(not_noisy_df_train[1:nb_noisy_train, :], noisy_df_train, kind=:outer, makeunique=true)
final_df_train = append!(noisy_df_train, not_noisy_df_train[1:nb_noisy_train, :])
final_df_test = append!(noisy_df_test, not_noisy_df_test[1:nb_noisy_test, :])
print("All datasets filled")
# TODO : See how it's possible to shuffler the whole dataframes
# py"shuffle"(final_df_train)
# py"shuffle"(final_df_test)

All datasets filled

In [12]:
x_dataset_train = Array{Float64,2}(final_df_train[2:nb_columns])
x_dataset_test = Array{Float64,2}(final_df_test[2:nb_columns])

y_dataset_train = Array{Int64,1}(final_df_train[1])
y_dataset_test = Array{Int64,1}(final_df_test[1])
print("Labels and data split")

Labels and data split

### Train part using SkLearn

In [13]:
using ScikitLearn
using ScikitLearn.GridSearch: GridSearchCV
@sk_import svm: SVC

┌ Warning: `getindex(o::PyObject, s::Symbol)` is deprecated in favor of dot overloading (`getproperty`) so elements should now be accessed as e.g. `o.s` instead of `o[:s]`.
│   caller = top-level scope at Skcore.jl:158
└ @ Core /home/jbuisine/.julia/packages/ScikitLearn/HK6Vs/src/Skcore.jl:158


PyObject <class 'sklearn.svm.classes.SVC'>

In [22]:
Cs = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
gammas = [0.001, 0.01, 0.1, 1, 5, 10, 100]
param_grid = Dict(:C => Cs, :gamma => gammas)

gridsearch = GridSearchCV(SVC(), param_grid)
fit!(gridsearch, x_dataset_train, y_dataset_train)
println("Best parameters: $(gridsearch.best_params_)")

Best parameters: Dict{Symbol,Any}(:gamma=>100.0,:C=>1000.0)


In [25]:
train_score = score(gridsearch, x_dataset_train, y_dataset_train)
test_score = score(gridsearch, x_dataset_test, y_dataset_test)
print("Train score : $(train_score)\n")
print("Test score : $(test_score)")

Train score : 0.9782244556113903
Test score : 0.5854341736694678